In [1]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
from newton_raphson import *

In [2]:
def get_grad_by_difference(symFunc, valueXk, delta):
    fk = symFunc.subs({x1: valueXk[0], x2: valueXk[1], x3: valueXk[2]})
    fk_x1 = f.subs({x1: valueXk[0] + delta, x2: valueXk[1], x3: valueXk[2]})
    fk_x2 = f.subs({x1: valueXk[0], x2: valueXk[1] + delta, x3: valueXk[2]})
    fk_x3 = f.subs({x1: valueXk[0], x2: valueXk[1], x3: valueXk[2] + delta})
    grad = np.array([(fk_x1 - fk)/delta, (fk_x2 - fk)/delta, (fk_x3 - fk)/delta])
    return grad

def find_optimal_alpha(symFunc, symVar):
    f_1st_derv = sym.diff(symFunc, symVar)
    f_2nd_derv = sym.diff(f_1st_derv, symVar)

    f_1st_derv = sym.lambdify(alpha, f_1st_derv, 'numpy')
    f_2nd_derv = sym.lambdify(alpha, f_2nd_derv, 'numpy')

    optValue, Iter = newton_raphson(f_1st_derv, f_2nd_derv, 0)
    return optValue, Iter

In [3]:
x1, x2, x3, alpha = sym.symbols('x1 x2 x3 alpha')
A = 3
B = 2
C = 2
f = A*x1**2 + x2**2 + B*x3**2 - 5*x1 + 2*x2
f # display f(x1, x2, x3)

3*x1**2 - 5*x1 + x2**2 + 2*x2 + 2*x3**2

In [4]:
maxIter = 100;
delta = 1.0e-05
epsilon = 1.0e-04

x = np.empty(shape=[0, 3])
x = np.append(x, [[C, B, A]], axis=0) # initial value
print(f'intial guess x0: {x}')

intial guess x0: [[2. 2. 3.]]


In [5]:
# Conjugate Gradient
for k in range(maxIter):
    # get gradient by finite difference
    grad = get_grad_by_difference(f, x[k], delta)
    
    if k == 0:
        direction = -grad;
    else:
        direction = -grad + (grad*np.transpose(grad)) / (pre_grad*np.transpose(pre_grad)) *pre_direction

    pre_grad = grad
    pre_direction = direction
    # find optimal alpha
    x_alpha = x[k] + alpha*direction
    f_alpha = f.subs({x1: x_alpha[0], x2: x_alpha[1], x3: x_alpha[2]})
    opt_alpha, Iter = find_optimal_alpha(f_alpha, alpha)
    
    # compute new x
    x_new = [x_alpha[0].subs(alpha, opt_alpha), x_alpha[1].subs(alpha, opt_alpha), x_alpha[2].subs(alpha, opt_alpha)]

    # save new x into array x
    x = np.append(x, [x_new], axis=0)
    
    # error check
    error = np.linalg.norm(x_new - x[k], 1)
    print(f'k: {k}, grad: {grad}, dir:{direction}, x: {x_new}, alpha: {opt_alpha}, error: {error}')
    if error < epsilon:
        print(x_new)
        break

k: 0, grad: [7.00002999991511 6.00000999995132 12.0000199999026], dir:[-7.00002999991511 -6.00000999995132 -12.0000199999026], x: [0.298298857709230, 0.541402841016822, 0.0828056820336438], alpha: 0.24309912133396663, error: 6.07749261924030
k: 1, grad: [-3.21017685374070 3.08281568200952 0.331242728135450], dir:[1.73800667269612 -4.66677179696948 -0.340386191641637], x: [0.859843630418761, -0.966417208920743, -0.0271720813556951], alpha: 0.32309701771076854, error: 2.17934258603644
k: 2, grad: [0.159091782503396 0.0671755821635145 -0.108668325404793], dir:[-0.154823147542232 -0.0693914523955353 0.0720342548693119], x: [0.826857159015083, -0.981201685250295, -0.0118245320644393], alpha: 0.213059041411619, error: 0.0631184970244860
k: 3, grad: [-0.0388270459339424 0.0376066294549560 -0.0472781282567780], dir:[0.0296053861478528 -0.0593542587620206 0.0609130819693010], x: [0.833687461005693, -0.994895393204272, 0.00222881427434034], alpha: 0.2307114643429503, error: 0.0345773562833654
k: